# Here We Download The Needed Data Set


In [3]:
!rm -r /content/sample_data/* #First we empty the sample data folder

In [4]:
# Here we download the csv file for the lookup for the taxi zones
!wget -P /content/sample_data "https://d37ci6vzurychx.cloudfront.net/misc/taxi+_zone_lookup.csv"

# Here we download the csv file for the weather data
!wget -O /content/sample_data/nyc-weather.zip "https://www.kaggle.com/api/v1/datasets/download/aadimator/nyc-weather-2016-to-2022" # Download The Required Dataset In Zip Format
!unzip /content/sample_data/nyc-weather.zip -d /content/sample_data # Then unzip The DataSet File
!rm /content/sample_data/nyc-weather.zip # And Then Remove The Zip File

# Create a loop to download each month's data from January to December
for month in range(1, 13):
    # Format the month as two digits (e.g., '01', '02', ..., '12')
    month_str = f"{month:02d}"
    # Construct the URL dynamically
    url = f"https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2017-{month_str}.parquet"
    # Download the file using wget
    !wget -P /content/sample_data {url}


--2024-11-20 12:23:16--  https://d37ci6vzurychx.cloudfront.net/misc/taxi+_zone_lookup.csv
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 54.230.209.200, 54.230.209.140, 54.230.209.72, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|54.230.209.200|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12331 (12K) [text/csv]
Saving to: ‘/content/sample_data/taxi+_zone_lookup.csv’

taxi+_zone_lookup.c 100%[===================>]  12.04K  --.-KB/s    in 0s      

2024-11-20 12:23:16 (24.7 MB/s) - ‘/content/sample_data/taxi+_zone_lookup.csv’ saved [12331/12331]

--2024-11-20 12:23:16--  https://www.kaggle.com/api/v1/datasets/download/aadimator/nyc-weather-2016-to-2022
Resolving www.kaggle.com (www.kaggle.com)... 35.244.233.98
Connecting to www.kaggle.com (www.kaggle.com)|35.244.233.98|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://storage.googleapis.com:443/kaggle-data-sets/

# Here We Set Up Spark Enviroment


In [5]:
!apt-get update # Update apt-get repository.
!apt-get install openjdk-8-jdk-headless -qq > /dev/null # Install Java.
!wget -q https://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz # Download Apache Sparks. Update the Spark version to 3.5.0
!tar xf spark-3.5.0-bin-hadoop3.tgz # Unzip the tgz file. Update the file name to match the downloaded file.
!pip install -q findspark # Install findspark. Adds PySpark to the System path during runtime.

# Set environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3" # Update the SPARK_HOME path to the new Spark version directory.

!ls

# Initialize findspark
import findspark
findspark.init()

# Install pyspark matching the spark version (3.5.0)
!pip install pyspark==3.5.0 # Install pyspark 3.5.0 to match Spark 3.5.0

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
sample_data  spark-3.5.0-bin-hadoop3  spark-3.5.0-bin-hadoop3.tgz


# Here We Start Our Spark Session

In [ ]:
# Create a PySpark session
from pyspark.sql import SparkSession
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder.master("local[1]").appName("NYC Taxi - BD Assesment Worex").getOrCreate()


# Here We Import Our Data Sets to Our Data Frames


In [8]:
# List of file paths
file_paths = [
    "/content/sample_data/yellow_tripdata_2017-01.parquet",
    "/content/sample_data/yellow_tripdata_2017-02.parquet",
    "/content/sample_data/yellow_tripdata_2017-03.parquet",
    "/content/sample_data/yellow_tripdata_2017-04.parquet",
    "/content/sample_data/yellow_tripdata_2017-05.parquet",
    "/content/sample_data/yellow_tripdata_2017-06.parquet",
    "/content/sample_data/yellow_tripdata_2017-07.parquet",
    "/content/sample_data/yellow_tripdata_2017-08.parquet",
    "/content/sample_data/yellow_tripdata_2017-09.parquet",
    "/content/sample_data/yellow_tripdata_2017-10.parquet",
    "/content/sample_data/yellow_tripdata_2017-11.parquet",
    "/content/sample_data/yellow_tripdata_2017-12.parquet"
]

# Read all Parquet & CSV files Into Data Frames
taxi_df_2017 = spark.read.parquet(*file_paths)
nyc_weather_df = spark.read.csv("/content/sample_data/NYC_Weather_2016_2022.csv", header=True, inferSchema=True)
taxi_zones_df = spark.read.csv("/content/sample_data/taxi+_zone_lookup.csv", header=True, inferSchema=True)
taxi_df_2017.printSchema()
nyc_weather_df.printSchema()
taxi_zones_df.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: integer (nullable = true)
 |-- airport_fee: integer (nullable = true)

root
 |-- time: timestamp (nullable = true)
 |-- temperature_2m (°C): double (nullable = true)
 |-- precipitation (mm): double (null

# Here We Select Our Needed Columns From Our DataSets

In [9]:
from pyspark.sql.functions import to_date

# Extract date from pickup datetime
#trip_data = taxi_df_2017.withColumn("pickup_date", to_date(taxi_df_2017["tpep_pickup_datetime"]))
trip_data = taxi_df_2017.select("tpep_pickup_datetime","tpep_dropoff_datetime", "PULocationID", "DOLocationID","passenger_count", "trip_distance","payment_type","fare_amount","extra", "total_amount")
trip_data = trip_data.withColumnRenamed("tpep_pickup_datetime", "pickup_datetime")
trip_data = trip_data.withColumnRenamed("tpep_dropoff_datetime", "dropoff_datetime")
zone_data = taxi_zones_df.select("LocationID", "Zone", "Borough")
weather_data = nyc_weather_df.select("time", "temperature_2m (°C)", "precipitation (mm)","rain (mm)","windspeed_10m (km/h)")


trip_data.show(5)
zone_data.show(5)
weather_data.show(5)




+-------------------+-------------------+------------+------------+---------------+-------------+------------+-----------+-----+------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|passenger_count|trip_distance|payment_type|fare_amount|extra|total_amount|
+-------------------+-------------------+------------+------------+---------------+-------------+------------+-----------+-----+------------+
|2017-01-01 00:32:05|2017-01-01 00:37:48|         140|         236|              1|          1.2|           2|        6.5|  0.5|         7.8|
|2017-01-01 00:43:25|2017-01-01 00:47:42|         237|         140|              2|          0.7|           2|        5.0|  0.5|         6.3|
|2017-01-01 00:49:10|2017-01-01 00:53:53|         140|         237|              2|          0.8|           2|        5.5|  0.5|         6.8|
|2017-01-01 00:36:42|2017-01-01 00:41:09|          41|          42|              1|          1.1|           2|        6.0|  0.5|         7.3|
|2017-

# Here We Join Our Taxi History Table To The Taxi Lookup Table

In [10]:
# Join trip data with zone data for pickup locations
trip_with_pickup_zones = trip_data.join(zone_data, trip_data.PULocationID == zone_data.LocationID, "left") \
                                  .withColumnRenamed("Zone", "Pickup_Zone") \
                                  .withColumnRenamed("Borough", "Pickup_Borough") \
                                  .withColumnRenamed("LocationID", "Pickup_LocationID")  # Rename LocationID to Pickup_LocationID
trip_with_pickup_zones.show(5)

# Join again for dropoff locations
trip_with_zones = trip_with_pickup_zones.join(zone_data, trip_with_pickup_zones.DOLocationID == zone_data.LocationID, "left") \
                                        .withColumnRenamed("Zone", "Dropoff_Zone") \
                                        .withColumnRenamed("Borough", "Dropoff_Borough") \
                                        .drop(zone_data.LocationID)  # Drop the LocationID from zone_data to avoid ambiguity
trip_with_zones.show(5,truncate=False)


+-------------------+-------------------+------------+------------+---------------+-------------+------------+-----------+-----+------------+-----------------+--------------------+--------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|passenger_count|trip_distance|payment_type|fare_amount|extra|total_amount|Pickup_LocationID|         Pickup_Zone|Pickup_Borough|
+-------------------+-------------------+------------+------------+---------------+-------------+------------+-----------+-----+------------+-----------------+--------------------+--------------+
|2017-01-01 00:32:05|2017-01-01 00:37:48|         140|         236|              1|          1.2|           2|        6.5|  0.5|         7.8|              140|     Lenox Hill East|     Manhattan|
|2017-01-01 00:43:25|2017-01-01 00:47:42|         237|         140|              2|          0.7|           2|        5.0|  0.5|         6.3|              237|Upper East Side S...|     Manhattan|
|2017-01-01 00:49:10

# Here We Calculate The Time Taken In Minutes & Seconds


In [11]:
trip_with_zones = trip_with_zones.withColumn(
    "trip_duration_seconds",
    unix_timestamp("dropoff_datetime") - unix_timestamp("pickup_datetime")
)
# Extract minutes and  seconds
trip_with_zones = trip_with_zones.withColumn(
    "trip_duration_minutes",
    (col("trip_duration_seconds") / 60).cast("integer")  # Get minutes
).withColumn(
    "trip_duration_seconds",
    col("trip_duration_seconds") % 60  # Get remaining seconds
)
trip_with_zones = trip_with_zones.withColumn("pickup_hour", date_trunc("hour", col("pickup_datetime")))
trip_with_zones.show(5)

+-------------------+-------------------+------------+------------+---------------+-------------+------------+-----------+-----+------------+-----------------+--------------------+--------------+----------+--------------------+---------------+---------------------+---------------------+-------------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|passenger_count|trip_distance|payment_type|fare_amount|extra|total_amount|Pickup_LocationID|         Pickup_Zone|Pickup_Borough|LocationID|        Dropoff_Zone|Dropoff_Borough|trip_duration_seconds|trip_duration_minutes|        pickup_hour|
+-------------------+-------------------+------------+------------+---------------+-------------+------------+-----------+-----+------------+-----------------+--------------------+--------------+----------+--------------------+---------------+---------------------+---------------------+-------------------+
|2017-01-01 00:32:05|2017-01-01 00:37:48|         140|         236|         

# And Then We Join It With The Weather DataSet

In [12]:
trip_final = trip_with_zones.join(weather_data, trip_with_zones.pickup_hour == weather_data.time, "left") \
                                   .drop("time").drop("pickup_hour")
trip_final.show(5)

+-------------------+-------------------+------------+------------+---------------+-------------+------------+-----------+-----+------------+-----------------+--------------------+--------------+----------+--------------------+---------------+---------------------+---------------------+-------------------+------------------+---------+--------------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|passenger_count|trip_distance|payment_type|fare_amount|extra|total_amount|Pickup_LocationID|         Pickup_Zone|Pickup_Borough|LocationID|        Dropoff_Zone|Dropoff_Borough|trip_duration_seconds|trip_duration_minutes|temperature_2m (°C)|precipitation (mm)|rain (mm)|windspeed_10m (km/h)|
+-------------------+-------------------+------------+------------+---------------+-------------+------------+-----------+-----+------------+-----------------+--------------------+--------------+----------+--------------------+---------------+---------------------+-------------------

# Here We Map The Payment Type Column To Its True Value


In [13]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Define a Python function for mapping
def map_payment_type(payment_type):
    mapping = {
        1: "Credit Card",
        2: "Cash",
        3: "No Charge",
        4: "Dispute",
        5: "Unknown",
        6: "Voided Trip"
    }
    return mapping.get(payment_type, "Other")

# Register the UDF
map_udf = udf(map_payment_type, StringType())

# Apply the UDF to create a new column
trip_final = trip_final.withColumn("payment_type", map_udf(col("payment_type")))

# Show results
trip_final.show(1)


+-------------------+-------------------+------------+------------+---------------+-------------+------------+-----------+-----+------------+-----------------+---------------+--------------+----------+--------------------+---------------+---------------------+---------------------+-------------------+------------------+---------+--------------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|passenger_count|trip_distance|payment_type|fare_amount|extra|total_amount|Pickup_LocationID|    Pickup_Zone|Pickup_Borough|LocationID|        Dropoff_Zone|Dropoff_Borough|trip_duration_seconds|trip_duration_minutes|temperature_2m (°C)|precipitation (mm)|rain (mm)|windspeed_10m (km/h)|
+-------------------+-------------------+------------+------------+---------------+-------------+------------+-----------+-----+------------+-----------------+---------------+--------------+----------+--------------------+---------------+---------------------+---------------------+------------

# Here We Chache The Final Data Frame

In [14]:
trip_final.cache() ## Here We Cache The Final Data Frame We Will Work On It

DataFrame[pickup_datetime: timestamp_ntz, dropoff_datetime: timestamp_ntz, PULocationID: bigint, DOLocationID: bigint, passenger_count: bigint, trip_distance: double, payment_type: string, fare_amount: double, extra: double, total_amount: double, Pickup_LocationID: int, Pickup_Zone: string, Pickup_Borough: string, LocationID: int, Dropoff_Zone: string, Dropoff_Borough: string, trip_duration_seconds: bigint, trip_duration_minutes: int, temperature_2m (°C): double, precipitation (mm): double, rain (mm): double, windspeed_10m (km/h): double]

# Here We Show The Most Payment Type People Pay With

In [ ]:
from pyspark.sql.functions import col, sum, count, round
# Aggregate data: Calculate total amount and count by payment type
total_amount_payment = (
    trip_final.groupBy("payment_type")
    .agg(
        sum("total_amount").alias("total_amount"),  # Sum of total_amount for each payment type
        count("payment_type").alias("payment_type_count")  # Count of each payment type
    )
    .withColumn("total_amount_millions", round(col("total_amount") / 1_000_000, 2))  # Convert total_amount to millions
    .orderBy(col("payment_type_count").desc())  # Sort in descending order by total_amount
)

# Show the results with only necessary columns
total_amount_payment.show()
